In [1]:
#Import necessary libraries
import numpy as np
import torch
from pkg_resources import packaging
import numpy as np
import matplotlib.pyplot as plt


print("Torch version:", torch.__version__)

from transformers import AutoImageProcessor, FlavaImageModel, FlavaModel, FlavaFeatureExtractor, AutoModelForCausalLM
from datasets import load_dataset




Torch version: 2.0.1+cu117


/home/IAIS/jraghu/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#check for GPU
torch.cuda.is_available()

True

In [3]:
#load the MNIST dataset using the load_dataset package
dataset = load_dataset("gorar/A-MNIST")

Found cached dataset a-mnist (/home/IAIS/jraghu/.cache/huggingface/datasets/gorar___a-mnist/amnist/1.1.0/49d6e25269c73523fbcc8d636818270c5604ddfdd1568ccabdcb39dc4416e954)
100%|██████████| 2/2 [00:00<00:00, 160.73it/s]


In [3]:
#oad CIFAR10 dataset also
cifar10 = load_dataset("cifar10")

Found cached dataset cifar10 (/home/IAIS/jraghu/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)
100%|██████████| 2/2 [00:00<00:00, 173.66it/s]


In [4]:
#Call the main pre-trained FLAVA model API's required for building upon the pre-trained model
#flava = FlavaModel.from_pretrained("facebook/flava-full").cuda().eval()
#fe = FlavaFeatureExtractor.from_pretrained("facebook/flava-full")

#Optional image model calls, can be used too
image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
model = FlavaImageModel.from_pretrained("facebook/flava-full").cuda().eval()


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Checking out Image processor class

Making the MNIST dataset smaller for testing purposes

In [9]:
train_small = dataset["train"]["image"]
train_small_label = dataset["train"]["label"]

In [10]:
feature_list = []
rgb_images = []
#target=(30,30)

#Iterate throught he MNIST dataset to convert it into 3 channel and append them into a list
for sample, label in zip(train_small, train_small_label):
  #sample =sample.resize(target)
  
  rgb_image = sample.convert('RGB')
  
  # Append the RGB image to the list
  rgb_images.append(rgb_image)
        

In [ ]:

# with torch.no_grad():
#     outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state

# last_hidden_states = last_hidden_states[:,0,:]

In [8]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 337461 KiB | 337461 KiB | 337461 KiB |      0 B   |
|       from large pool | 336384 KiB | 336384 KiB | 336384 KiB |      0 B   |
|       from small pool |   1077 KiB |   1077 KiB |   1077 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         | 337461 KiB | 337461 KiB | 337461 KiB |      0 B   |
|       from large pool | 336384 KiB | 336384 KiB | 336384 KiB |

In [5]:
#BAtching the  dataset to get image features in batches to enable the efficeint and complete use of the GPU

def batching(batch_size, rgb_images):
    """
    Batch_size-> is the number o batches you want to process the dataset in; rgb_images-> the dataset list in RGB form
    returns a list of features of lists in tensors
    """

    # Define the batch size
    batch_size = 10

    features_in_Tensor = []
    # Calculate the number of batches
    num_batches = len(rgb_images) // batch_size

    # Process the images in batches
    for batch_idx in range(num_batches):
        # Get the start and end indices for the current batch
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size

        # Get the images for the current batch
        batch_images = rgb_images[start_idx:end_idx]

        # Process the images in the current batch
        processed_images = []
        with torch.no_grad():
            for image in batch_images:
                # Your processing logic here

                #processed_images = fe(image, return_tensors="pt").to("cuda")
                #image_features = flava.get_image_features(**processed_images)[:, 0, :]
                processed_images = image_processor(image, return_tensors="pt").to("cuda")
                image_features = model(**processed_images)
                image_features = image_features.last_hidden_state[:, 0, :]
                
                features_in_Tensor.append(image_features.detach().cpu().numpy())
        # Do something with the processed images
        
        
        # Clear memory of the processed images if no longer needed
        torch.cuda.empty_cache()
        del processed_images, image_features

    return features_in_Tensor

features_in_Tensor = batching(batch_size=10, rgb_images=rgb_images)


Since the FLAVA paper mentions that they use a L-BFGS based logistic regression classifier as a classifier head on top of the image encoder to use the mebeddings, I have implemented the same from the same source as mentioned in the paper and further ahead is the results.

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#Converting the tensor into Numpy 
#features = features_in_Numpy.detach().numpy()
features_in_Numpy = np.squeeze(features_in_Tensor, axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_in_Numpy, train_small_label, test_size=0.2, random_state=42)

# Create a logistic regression model with L-BFGS optimization
logistic_model = LogisticRegression(solver='lbfgs', max_iter=1000)

# Train the logistic regression model
logistic_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logistic_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("training Accuracy:", accuracy)

Accuracy: 0.970625


In [58]:
print(np.shape(X_train), np.shape(X_test), np.shape(y_train),np.shape(y_test))

(96000, 768) (24000, 768) (96000,) (24000,)


In [21]:
#Preprocessing the test dataset to covert in ndarray
test_small = dataset["test"]["image"][0:1000]
test_small_label = dataset["test"]["label"][0:1000]

rgb_images_test = []


#Iterate throught he MNIST dataset to convert it into 3 channel and append them into a list
for sample, label in zip(test_small, test_small_label):
  
  
  rgb_image = sample.convert('RGB')
  
  # Append the RGB image to the list
  rgb_images_test.append(rgb_image)


features_test = batching(batch_size=10, rgb_images=rgb_images_test)
features_test = np.squeeze(features_test, axis=1)
# Make predictions on the unseen dataset using the same Logistic regression linear classifier
y_pred_unseen = logistic_model.predict(features_test)

# Calculate accuracy
accuracy = accuracy_score(test_small_label, y_pred_unseen)
print("test set Accuracy:", accuracy)

Accuracy: 0.938


CIFAR-10 with FLAVA-- Doing the whole same procedure as done above for MNISt for CIFAR-10 dataset

In [7]:
train_cifar = cifar10["train"]["img"]
train_label_cifar = cifar10["train"]["label"]

In [8]:
feature_list_cifar = []
rgb_images_cifar = []

for sample, label in zip(train_cifar, train_label_cifar):
  # Convert to RGB
  rgb_image = sample.convert('RGB')
      
  # Append the RGB image to the list
  rgb_images_cifar.append(rgb_image)

In [9]:

#Calling the batching function with parameters
features_in_Tensor = batching(batch_size=10, rgb_images=rgb_images_cifar)

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#Restructing the features in 2D format
features_in_Numpy = np.squeeze(features_in_Tensor, axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_in_Numpy, train_label_cifar, test_size=0.2, random_state=42)

# Create a logistic regression model with L-BFGS optimization
logistic_model = LogisticRegression(solver='lbfgs', max_iter=1000)

# Train the logistic regression model
logistic_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logistic_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Train set Accuracy for CIFAR10:", accuracy)

Train set Accuracy for CIFAR10: 0.962


In [11]:
#Preprocessing the test dataset to covert in ndarray
test_cifar = cifar10["test"]["img"]
test_label_cifar = cifar10["test"]["label"]

rgb_images_test_cifar10 = []


#Iterate throught he MNIST dataset to convert it into 3 channel and append them into a list
for sample, label in zip(test_cifar, test_label_cifar):
  
  
  rgb_image = sample.convert('RGB')
  
  # Append the RGB image to the list
  rgb_images_test_cifar10.append(rgb_image)


features_test = batching(batch_size=10, rgb_images=rgb_images_test_cifar10)
features_test = np.squeeze(features_test, axis=1)
# Make predictions on the unseen dataset using the same Logistic regression linear classifier
y_pred_unseen = logistic_model.predict(features_test)

# Calculate accuracy
accuracy = accuracy_score(test_label_cifar, y_pred_unseen)
print("Test set Accuracy for CIFAR10:", accuracy)

Test set Accuracy for CIFAR10: 0.9589
